# In this notebook we are going to analyse the data scrapped from jumia.cm

In [2]:
# importing modules
import numpy as np
import pandas as pd
import re 
%matplotlib inline
import matplotlib.pyplot as plt

In [19]:
# loading the scrapped data
df = pd.read_csv('./dataframe.csv')
#we have a column called  Unnamed so it will be good to drop it before continuing
#df.drop(['unnamed'], axis='columns')
print(df.info())
print(df.seller=="Rock")
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
Unnamed: 0              918 non-null int64
name                    918 non-null object
seller                  918 non-null object
price                   918 non-null float64
number-of-items-sold    871 non-null float64
ram                     789 non-null float64
color                   916 non-null object
screen-size             771 non-null float64
rom                     815 non-null float64
front-camera            631 non-null float64
rear-camera             631 non-null float64
number_of_sims          918 non-null int64
dtypes: float64(7), int64(2), object(3)
memory usage: 86.1+ KB
None
0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     F

,Unnamed: 0,price,number-of-items-sold,ram,screen-size,rom,front-camera,rear-camera,number_of_sims
count,918.000000,9.180000e+02,871.000000,789.000000,771.000000,815.000000,631.000000,631.000000,918.000000
mean,458.500000,1.794193e+05,1874.060850,8.092522,5.490532,49.023313,6.194453,11.598415,1.477124
std,265.148072,1.687216e+05,9219.924159,51.086100,0.783400,91.981387,3.941934,3.785523,0.499749
min,0.000000,1.039000e+03,9.000000,1.000000,0.000000,4.000000,0.000000,0.300000,1.000000
25%,229.250000,7.500000e+04,49.000000,1.000000,5.000000,16.000000,5.000000,8.000000,1.000000
50%,458.500000,1.364750e+05,101.000000,3.000000,5.500000,32.000000,5.000000,12.000000,1.000000
75%,687.750000,2.155000e+05,1001.000000,4.000000,6.000000,64.000000,8.000000,13.000000,2.000000
max,917.000000,2.210000e+06,100001.000000,512.000000,12.000000,1616.000000,24.000000,40.000000,2.000000
